<a href="https://colab.research.google.com/github/Tanya-Verma/Apline_dashboard/blob/main/Copy_of_GPT_Dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import torch # we use PyTorch: https://pytorch.org

!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print("length of dataset in characters: ", len(text))

print(text[:1000])

chars = sorted(list(set(text)))

stoi = { ch:i for i,ch in enumerate(chars) }

--2026-02-02 07:23:07--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.2’

input.txt.2         100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2026-02-02 07:23:07 (21.4 MB/s) - ‘input.txt.2’ saved [1115394/1115394]

length of dataset in characters:  1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a

## Implement `get_batch` Function

### Subtask:
Implement the `get_batch` function, which will randomly sample batches of data from either the training or validation set. This function is crucial for efficiently feeding data to the model during training.

In [10]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

# Test the get_batch function
xb, yb = get_batch('train')
print("Inputs (xb) shape:", xb.shape)
print("Targets (yb) shape:", yb.shape)
print("First batch inputs:", xb[0])
print("First batch targets:", yb[0])

Inputs (xb) shape: torch.Size([4, 8])
Targets (yb) shape: torch.Size([4, 8])
First batch inputs: tensor([57,  6,  0, 13,  1, 44, 53, 56])
First batch targets: tensor([ 6,  0, 13,  1, 44, 53, 56, 58])


# Task
Define the `encode` and `decode` functions, set `block_size` and `batch_size`, implement `get_batch`, define `B`, `T`, `C`, `head_size`, and `xbow2` for toy examples, and provide a dummy PyTorch model (`m`) and optimizer to make the existing code snippets runnable.

## Define Tokenization Functions

### Subtask:
Implement the `encode` and `decode` functions based on the `stoi` (string to integer) and `itos` (integer to string) mappings, which are crucial for processing the text data.


**Reasoning**:
The subtask requires defining `itos`, `encode`, and `decode` functions. I will start by creating the `itos` dictionary from the existing `stoi` dictionary and then implement the `encode` and `decode` functions as specified.



In [5]:
itos = { i:ch for i,ch in enumerate(chars) }

def encode(s):
    return [stoi[c] for c in s] # encoder: take a string, output a list of integers

def decode(l):
    return ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print("itos dictionary created successfully.")
print("encode function defined.")
print("decode function defined.")

data = torch.tensor(encode(text), dtype=torch.long)
print(f"Data tensor created with shape: {data.shape}")

itos dictionary created successfully.
encode function defined.
decode function defined.
Data tensor created with shape: torch.Size([1115394])


## Set Hyperparameters and Split Data

### Subtask:
Define `block_size` and `batch_size`. Split the `data` tensor into training and validation sets based on a 90/10 ratio.


**Reasoning**:
The subtask requires defining `block_size` and `batch_size`, and splitting the `data` tensor into training and validation sets. I will create a code block to accomplish these steps as per the instructions.



In [6]:
block_size = 8 # what is the maximum context length for predictions?
batch_size = 4 # how many independent sequences will we process in parallel?

n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

print(f"block_size set to: {block_size}")
print(f"batch_size set to: {batch_size}")
print(f"Training data length: {len(train_data)}")
print(f"Validation data length: {len(val_data)}")

block_size set to: 8
batch_size set to: 4
Training data length: 1003854
Validation data length: 111540
